In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d roobansappani/hand-gesture-recognition

Dataset URL: https://www.kaggle.com/datasets/roobansappani/hand-gesture-recognition
License(s): copyright-authors
100% 27.0M/27.1M [00:01<00:00, 35.4MB/s]
100% 27.1M/27.1M [00:01<00:00, 22.5MB/s]


In [ ]:
!unzip hand-gesture-recognition.zip -d /content/dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/HandGesture/images/call_me/3992.jpg  
  inflating: /content/dataset/HandGesture/images/call_me/3999.jpg  
  inflating: /content/dataset/HandGesture/images/call_me/4006.jpg  
  inflating: /content/dataset/HandGesture/images/call_me/4012.jpg  
  inflating: /content/dataset/HandGesture/images/call_me/4020.jpg  
  inflating: /content/dataset/HandGesture/images/call_me/4028.jpg  
  inflating: /content/dataset/HandGesture/images/call_me/4035.jpg  
  inflating: /content/dataset/HandGesture/images/call_me/4043.jpg  
  inflating: /content/dataset/HandGesture/images/call_me/4049.jpg  
  inflating: /content/dataset/HandGesture/images/call_me/4057.jpg  
  inflating: /content/dataset/HandGesture/images/call_me/4066.jpg  
  inflating: /content/dataset/HandGesture/images/call_me/411.jpg  
  inflating: /content/dataset/HandGesture/images/call_me/425.jpg  
  inflating: /content/dataset/HandGesture/images/call_me/4327.jpg  

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [15]:
# Set paths and parameters
base_path = "/content/dataset/HandGesture/images"  # Update this path
img_size = (64, 64)
batch_size = 32

# Create an ImageDataGenerator for loading images
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalize pixel values and split for validation

train_generator = datagen.flow_from_directory(
    base_path,
    target_size=img_size,
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Set as training data
)

validation_generator = datagen.flow_from_directory(
    base_path,
    target_size=img_size,
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

Found 4199 images belonging to 10 classes.
Found 1044 images belonging to 10 classes.


In [16]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
from tensorflow.keras.callbacks import EarlyStopping
# Train the Model with Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(train_generator, epochs=10, validation_data=validation_generator, callbacks=[early_stopping])

Epoch 1/10
132/132 [==============================] - 26s 201ms/step - loss: 0.0641 - accuracy: 0.9829 - val_loss: 0.4094 - val_accuracy: 0.8755
Epoch 2/10
132/132 [==============================] - 24s 180ms/step - loss: 0.0322 - accuracy: 0.9905 - val_loss: 0.3912 - val_accuracy: 0.8851
Epoch 3/10
132/132 [==============================] - 26s 198ms/step - loss: 0.0166 - accuracy: 0.9955 - val_loss: 0.4423 - val_accuracy: 0.9004
Epoch 4/10
132/132 [==============================] - 29s 221ms/step - loss: 0.0165 - accuracy: 0.9962 - val_loss: 0.4448 - val_accuracy: 0.8927
Epoch 5/10
132/132 [==============================] - 22s 168ms/step - loss: 0.0053 - accuracy: 0.9990 - val_loss: 0.4670 - val_accuracy: 0.9004
Epoch 6/10
132/132 [==============================] - 26s 197ms/step - loss: 0.0061 - accuracy: 0.9988 - val_loss: 0.5295 - val_accuracy: 0.8870
Epoch 7/10
132/132 [==============================] - 24s 180ms/step - loss: 0.0326 - accuracy: 0.9888 - val_loss: 0.5483 - val_ac

In [20]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

33/33 [==============================] - 2s 48ms/step - loss: 0.3912 - accuracy: 0.8851
Validation Loss: 0.39116543531417847
Validation Accuracy: 0.8850574493408203


In [22]:
model.save("hand_gesture_model.keras")